In [1]:
# Set our python libraries to dynamically reload so that we can edit
# the external text file while calling the function in the notebook
# without having to restart the kernel
%load_ext autoreload
%autoreload 2

In [2]:
# Import all of the functions from our custom functions library
from function_library import *

# Goal

In this project, we aim to be able to predict the housing prices in King County using linear regression.

# Data overview

View data descriptions

In [4]:
! cat column_names.md

# Column Names and descriptions for Kings County Data Set
* **id** - unique identified for a house
* **dateDate** - house was sold
* **pricePrice** -  is prediction target
* **bedroomsNumber** -  of Bedrooms/House
* **bathroomsNumber** -  of bathrooms/bedrooms
* **sqft_livingsquare** -  footage of the home
* **sqft_lotsquare** -  footage of the lot
* **floorsTotal** -  floors (levels) in house
* **waterfront** - House which has a view to a waterfront
* **view** - Has been viewed
* **condition** - How good the condition is ( Overall )
* **grade** - overall grade given to the housing unit, based on King County grading system
* **sqft_above** - square footage of house apart from basement
* **sqft_basement** - square footage of the basement
* **yr_built** - Built Year
* **yr_renovated** - Year when house was renovated
* **zipcode** - zip
* **lat** - Latitude coordinate
* **long** - Longitude coordinate
* **sqft_living15** - The square footage of interior housing living space for the neares

# Data cleaning

# Deriving new features

# Data transformation

# Data normalization

# Checking for outliers

# Linear regression assumptions

# Feature selection

# Model building

# Model evaluation

# Interpretation of coefficients in regression

# Influence of different modeling choices on prediction accuracy

# Log transform of target variable?